In [ ]:
#import packages
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import json

In [ ]:
#data import
game_data = pd.read_csv('data/clean_data.csv')


In [ ]:
sb.heatmap(data=game_data.corr())


In [ ]:
corr_table = game_data.corr()["Price"]
corr_table.head()

In [ ]:
for index, row in corr_table.items():
    print(index, " corr:", row)

In [ ]:
price_tag = pd.DataFrame(game_data[["AppID", "Name", "Price", "Tags"]])
price_category = pd.DataFrame(game_data[["AppID", "Name", "Price", "Categories"]])
price_genre = pd.DataFrame(game_data[["AppID", "Name", "Price", "Genres"]])

In [ ]:
def seperate(df, column):
    json_table = []
    json_file = df.to_json(orient='records')
    json_file = json.loads(json_file)

    for j in json_file:
        test = {}
        temp = j[column]
        test["AppID"] = j["AppID"]
        test["Name"] = j["Name"]
        test["Price"] = j["Price"]
        
        if temp != None:
            temp = temp.split(",")
            for i in temp:
                test[i] = 1
        
        json_table.append(test)

    res_table = pd.DataFrame(json_table)
    print(res_table.shape)
    print(res_table.columns.tolist())
    res_table.fillna(value=0, inplace=True)

    print(len(json_table))
    return res_table

In [ ]:
price_tag_sep = seperate(price_tag, "Tags")
price_category_sep = seperate(price_category, "Categories")
price_genre_sep = seperate(price_genre, "Genres")

tags = price_tag_sep.columns.tolist()
categories = price_category_sep.columns.tolist()
genres = price_genre_sep.columns.tolist()

In [ ]:
tags = tags[3:]
categories = categories[3:]
genres = genres[3:]
print(tags)
print(categories)
print(genres)
print(len(tags), len(categories), len(genres))

In [ ]:
collection = {}
for i in tags:
    if(i not in collection.keys()):
        collection[i] = ["Tags"]

for i in categories:
    if(i not in collection.keys()):
        collection[i] = ["Categories"]
    else:
        collection[i].append("Categories")

for i in genres:
    if(i not in collection.keys()):
        collection[i] = ["Genres"]
    else:
        collection[i].append("Genres")

print(collection)
len(collection)

In [ ]:
new = pd.DataFrame()
for i in collection.keys():
    if(len(collection[i]) > 1):
        print("here: ", i ,collection[i])
        if(collection[i] == ["Tags", "Categories"]):
            new[i] = np.ceil((price_tag_sep[i] + price_category_sep[i])/2)
        elif(collection[i] == ["Tags", "Genres"]):
            new[i] = np.ceil((price_tag_sep[i] + price_genre_sep[i])/2)
        elif(collection[i] == ["Categories", "Genres"]):
            new[i] = np.ceil((price_category_sep[i] + price_genre_sep[i])/2)
    else:
        print("aaa: ", i ,collection[i])
        if(collection[i] == ["Tags"]):
            new[i] = price_tag_sep[i]
        elif(collection[i] == ["Categories"]):
            new[i] = price_category_sep[i]
        elif(collection[i] == ["Genres"]):
            new[i] = price_genre_sep[i]

new.head(20)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [ ]:
test_regr = RandomForestRegressor(n_estimators=200, min_samples_split=5, max_features=0.2)
# max_leaf_nodes=100, max_depth=25, 
X = new
Y = game_data["Price"]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

test_regr.fit(X_train, Y_train)
maybe = test_regr.predict(X_train)
maybe2 = test_regr.predict(X_test)
mse = mean_squared_error(Y_train, maybe)
mse2 = mean_squared_error(Y_test, maybe2)
score = test_regr.score(X_train, Y_train)
score2 = test_regr.score(X_test, Y_test)
print("train score: ",score, " mse: ", mse,  mse**(1/2))
print("test score: ",score2, " mse: ", mse2,  mse2**(1/2))

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(24, 12))
axes[0].scatter(Y_train, maybe, color = "blue")
axes[0].plot(Y_train, Y_train, 'w-', linewidth = 1)
axes[0].set_xlabel("True values of the Response Variable (Train)")
axes[0].set_ylabel("Predicted values of the Response Variable (Train)")
axes[1].scatter(Y_test, maybe2, color = "green")
axes[1].plot(Y_test, Y_test, 'w-', linewidth = 1)
axes[1].set_xlabel("True values of the Response Variable (Test)")
axes[1].set_ylabel("Predicted values of the Response Variable (Test)")
plt.show()

In [ ]:
#adding more variables
round2 = pd.DataFrame(new)
round2["Metacritic score"] = game_data["Metacritic score"]
round2["Peak CCU"] = game_data["Peak CCU"]
round2["Recommendations"] = game_data["Recommendations"]
round2.head()


In [ ]:
# test_regr = RandomForestRegressor(max_depth=25, n_estimators=200, min_samples_split=5, max_features=0.4)
test_regr = RandomForestRegressor(n_estimators=200, max_depth=20, max_features=0.4)

# max_leaf_nodes=100, max_depth=25,

X = round2
Y = game_data["Price"]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25)

test_regr.fit(X_train, Y_train)
maybe = test_regr.predict(X_train)
maybe2 = test_regr.predict(X_test)
mse = mean_squared_error(Y_train, maybe)
mse2 = mean_squared_error(Y_test, maybe2)
score = test_regr.score(X_train, Y_train)
score2 = test_regr.score(X_test, Y_test)
print("train score: ",score, " mse: ", mse,  mse**(1/2))
print("test score: ",score2, " mse: ", mse2,  mse2**(1/2))

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(24, 12))
axes[0].scatter(Y_train, maybe, color = "blue")
axes[0].plot(Y_train, Y_train, 'w-', linewidth = 1)
axes[0].set_xlabel("True values of the Response Variable (Train)")
axes[0].set_ylabel("Predicted values of the Response Variable (Train)")
axes[1].scatter(Y_test, maybe2, color = "green")
axes[1].plot(Y_test, Y_test, 'w-', linewidth = 1)
axes[1].set_xlabel("True values of the Response Variable (Test)")
axes[1].set_ylabel("Predicted values of the Response Variable (Test)")
plt.show()